# Extract a dictionary from MFA-aligned TextGrids

> "Need to re-align, don't have the dictionary"

- toc: false
- branch: master
- comments: true
- categories: [textgrid, mfa]

In [1]:
%%capture
!pip install praatio

In [2]:
from praatio import textgrid

In [3]:
def get_combined_words_and_phones(filename):
    tg = textgrid.openTextgrid(filename, False)
    if not tg.tierNameList or tg.tierNameList != ['Word', 'phones']:
        return []
    
    word = tg.tierDict['Word']
    phones = tg.tierDict['phones']
    
    i = 0
    j = 0
    out = []

    def it_to_dict(it):
        ret = {}
        ret['start'] = it.start
        ret['end'] = it.end
        ret['label'] = it.label
        return ret

    while i < len(word.entryList) and j < len(phones.entryList):
        cur_word = it_to_dict(word.entryList[i])
        cur_word['phones'] = []
        while j < len(phones.entryList) and phones.entryList[j].end <= cur_word['end']:
            end_time = phones.entryList[j].end
            tmp_phone = it_to_dict(phones.entryList[j])
            cur_word['phones'].append(tmp_phone)
            j += 1
            if end_time == cur_word['end']:
                i += 1
                out.append(cur_word)
                continue
    return out

In [4]:
def get_wordlist_from_combined(items):
    tmp = []
    for item in items:
        word = item['label']
        phones = " ".join([a['label'] for a in item['phones']])
        if phones == "sil":
            continue
        tmp.append((word, phones))
    return tmp